In [1]:
import glob
import tensorflow as tf

print(tf.__version__)

2.6.0


# Step 2. 데이터 읽어오기

In [2]:
import glob
import os
import pandas
import matplotlib.pyplot as plt

txt_file_path = './data/lyrics/*'
txt_list = glob.glob(txt_file_path)
raw_corpus = []

for txt_file in txt_list:
    with open(txt_file,"r") as f:
        raw = f.read().splitlines()
        raw_corpus.extend(raw)

print("데이터 크기 : ",len(raw_corpus))
print("Examples :\n",raw_corpus[:3])        

데이터 크기 :  187088
Examples :
 ['', '', 'All of this and more is for you']


In [3]:
#토큰의 개수가 15개가 넘는 문장들은 삭제하기 위해 문장들의 길이 파악
#for i in range(len(raw_corpus)):
#    print(len(raw_corpus[i]))
#print("Length of each corpus :\n",len(raw_corpus))
#토큰화를 해보고 나서 너무 긴 문장들은 삭제하는 함수를 써주자. 

0
0
31
37
24
0
0
26
33
27
32
31
29
27
24
0
27
33
0
33
35
9
43
9
38
27
0
26
35
25
28
27
23
33
22
0
27
33
0
33
34
9
43
9
38
27
0
19
33
9
43
9
42
27
0
24
9
37
9
43
38
0
7
7
35
28
7
7
0
0
29
25
27
32
40
9
53
45
47
44
37
27
0
73
61
0
12
12
0
12
12
0
50
39
29
0
48
38
54
26
24
11
8
21
0
21
32
25
12
0
0
36
41
51
0
48
43
49
0
0
45
33
43
35
31
0
20
28
30
44
0
53
32
39
33
28
44
0
30
31
30
44
0
28
40
27
41
0
30
30
30
30
30
30
0
36
0
0
27
23
27
33
0
31
36
46
32
0
27
34
27
44
0
47
29
52
38
0
16
38
35
40
0
0
24
33
14
22
32
0
28
42
31
49
10
0
42
47
42
52
0
28
30
26
52
10
0
41
32
33
29
47
0
42
43
42
43
0
0
25
26
25
35
0
30
46
39
37
0
24
20
33
31
0
38
38
31
0
42
31
11
0
0
23
22
43
27
35
0
22
35
20
0
34
31
45
28
0
27
22
25
31
0
34
31
45
17
9
0
0
20
45
20
36
21
30
31
31
33
27
21
51
21
65
15
36
0
23
43
20
42
21
37
29
28
38
27
0
21
51
21
64
15
42
0
0
35
33
31
29
25
23
33
31
35
32
27
46
0
29
25
25
21
29
25
0
23
21
27
33
28
30
35
45
0
35
33
28
41
0
29
25
25
21
29
25
0
0
11
11
38
39
33
37
24
24
0
45
18
45
18
4

24
9
24
9
0
29
17
14
29
11
19
0
20
21
13
47
21
28
0
9
9
9
12
0
24
54
28
37
0
9
9
9
9
13
0
0
4
6
0
31
25
24
31
35
0
16
26
16
22
0
33
37
33
27
36
0
16
26
16
22
0
7
4
3
5
0
33
31
22
24
35
0
16
26
16
22
0
16
26
16
22
0
7
4
3
5
0
22
11
22
17
43
0
0
32
31
32
36
22
0
42
61
54
24
24
6
0
8
24
8
25
8
25
28
24
6
0
42
53
57
33
28
6
0
8
24
8
25
8
25
28
24
6
0
28
65
28
45
19
6
0
8
24
8
25
8
28
28
24
6
0
8
24
8
25
8
25
28
24
6
0
0
42
42
30
22
40
0
44
41
30
20
33
0
32
37
33
23
21
0
45
31
70
21
41
42
0
33
41
44
58
0
32
37
71
20
23
28
0
68
15
61
57
64
30
52
50
0
10
68
77
11
71
17
45
46
19
0
64
9
53
54
63
33
18
22
0
0
3
10
4
11
0
38
53
57
61
16
30
0
10
47
10
44
0
22
38
19
61
43
32
42
41
0
10
47
10
44
0
34
0
17
0
10
47
10
44
0
10
49
0
10
67
54
19
59
22
69
46
7
46
12
26
9
0
40
41
0
5
0
19
0
7
0
21
0
10
13
20
32
17
16
30
8
0
0
47
64
59
71
0
54
63
0
47
43
55
55
26
0
0
27
15
34
25
31
35
68
0
36
32
42
26
72
72
7
0
25
0
46
22
30
43
33
42
49
34
39
31
0
0
14
12
12
14
12
12
0
15
12
12
19
12
18
0
14
12
12
14
12
12


16
26
8
5
12
0
36
35
35
14
35
28
41
0
12
24
26
0
19
30
26
11
22
18
17
43
58
31
12
25
0
18
22
18
28
12
18
30
22
26
0
17
51
35
30
9
0
22
6
22
6
29
5
26
5
29
5
25
5
14
0
34
15
10
21
19
31
23
0
27
0
18
5
22
4
18
5
22
4
11
12
6
18
12
6
18
26
22
22
11
18
19
19
18
18
36
19
0
55
0
30
7
6
5
0
0
9
49
39
0
38
44
45
56
33
18
5
0
31
30
31
28
31
38
31
30
0
62
64
0
31
29
37
39
31
32
31
45
0
60
62
19
0
48
25
58
44
42
21
30
0
31
40
23
29
29
29
29
29
29
29
16
31
45
40
47
41
23
0
34
28
28
0
13
31
31
31
59
5
0
0
27
22
34
0
42
57
54
67
0
53
57
58
38
41
47
0
57
59
59
57
0
53
57
58
38
41
47
0
41
21
19
0
76
35
50
68
75
0
53
57
58
38
41
47
32
0
40
30
29
41
51
0
56
65
68
30
30
0
0
37
46
38
31
0
35
58
45
31
0
28
28
49
18
49
37
34
0
18
29
34
23
21
0
30
49
48
21
56
42
16
25
0
11
41
45
0
28
36
54
33
32
32
45
45
45
37
36
34
11
0
49
42
45
54
62
41
28
43
44
45
33
46
50
0
37
31
35
40
78
0
0
41
34
42
41
37
46
0
48
33
39
0
2
0
0
43
50
48
63
41
39
0
69
60
55
58
45
31
0
72
77
49
47
0
72
45
33
63
38
33
35
46
0
43
58
11
0
72

43
0
38
18
43
49
40
18
68
37
14
30
51
35
22
34
43
68
0
0
20
52
17
55
25
27
41
22
37
27
8
0
30
27
17
38
50
14
6
44
41
55
25
0
25
0
37
18
33
16
24
30
32
41
22
23
3
16
1
39
0
0
24
25
20
25
0
68
10
72
10
69
10
46
32
0
65
10
71
10
74
10
56
32
0
68
46
59
0
0
39
49
44
29
32
48
39
43
59
38
76
50
27
42
62
55
58
0
63
33
0
45
49
17
53
34
35
26
21
46
38
22
42
29
41
23
27
38
31
33
31
35
29
24
40
25
0
62
60
62
50
0
15
0
18
18
18
22
22
19
28
22
19
28
22
0
0
31
29
26
36
28
17
29
31
32
22
31
29
26
36
28
17
29
31
32
22
0
36
34
35
63
35
0
38
45
40
43
36
39
0
31
29
26
36
28
17
29
31
32
22
0
16
0
45
47
30
36
51
0
45
41
49
37
14
32
0
31
29
26
36
28
17
29
31
32
22
0
14
14
14
0
0
22
28
38
35
21
34
40
35
37
21
0
48
63
44
37
29
0
5
23
0
25
34
65
33
29
0
26
36
47
33
21
0
19
27
17
26
32
32
19
0
0
48
43
44
36
31
44
35
35
13
13
0
37
27
36
28
45
32
37
42
42
31
0
25
26
31
30
0
25
26
31
30
0
48
37
0
25
26
21
19
0
40
19
18
50
26
0
18
21
21
21
21
0
25
19
0
25
0
18
0
0
18
46
29
26
31
25
26
6
25
0
0
18
38
0
34
0
14
0
63
4

23
26
29
22
23
24
19
20
24
0
0
0
0
0
3
0
31
23
35
23
0
29
29
38
25
0
26
26
35
23
0
35
30
31
19
0
35
25
33
30
0
0
0
0
0
4
0
14
0
31
28
32
25
0
31
28
30
20
0
0
0
0
0
5
0
6
0
23
19
17
22
0
24
12
18
26
0
29
11
15
12
0
0
0
0
0
3
0
42
27
36
23
0
36
24
35
23
0
44
25
43
25
0
31
30
38
25
0
0
0
0
0
2
0
13
0
32
19
34
21
24
21
37
28
31
21
30
25
0
0
0
0
0
3
0
11
0
22
26
0
21
24
0
22
21
0
9
8
0
0
0
0
0
4
0
8
0
23
17
22
16
0
28
10
24
10
0
27
17
23
15
0
25
17
32
14
0
23
19
26
18
0
26
17
21
14
0
22
19
24
16
0
22
32
13
12
0
28
22
21
12
0
30
14
12
21
0
19
21
19
26
0
22
18
23
16
11
25
8
0
0
0
0
0
5
0
13
0
109
0
0
33
16
40
21
0
32
30
33
27
0
39
20
29
25
0
35
31
24
22
0
35
31
32
24
0
37
23
32
22
0
37
31
31
26
0
0
0
0
0
4
0
15
0
37
34
34
22
34
38
28
0
35
35
20
40
31
22
0
36
39
25
31
33
29
0
0
0
0
0
3
0
13
0
35
23
32
24
0
27
19
34
22
0
24
21
31
21
0
26
21
25
24
0
0
0
0
0
4
0
11
0
29
17
24
14
0
29
25
27
23
0
24
25
16
21
0
0
0
0
0
5
0
9
0
36
26
26
21
0
34
21
33
19
0
30
24
28
23
0
0
0
0
0
6
0
11
0
18
20
26
20
0


...

22
53
28
60
103
31
48
38
56
24
27
64
37
65
101
31
48
38
56
24
27
41
30
18
43
41
101
31
48
38
56
24
27
24
91
10
32
32
27
32
22
26
25
34
119
59
56
59
103
24
46
24
46
24
74
32
27
32
22
26
25
34
20
20
20
20
148
56
65
65
47
24
47
24
47
24
74
26
26
26
56
89
21
34
20
20
35
40
20
45
22
37
47
93
69
30
38
89
21
34
20
19
35
40
20
45
22
37
47
93
69
30
101
26
23
22
30
26
23
22
20
47
22
37
58
93
69
30
97
69
30
102
65
33
33
56
25
27
25
22
28
27
30
20
26
15
21
18
53
27
21
34
18
37
29
25
35
9
24
25
27
9
43
33
33
60
28
23
26
15
23
52
20
26
15
21
18
48
27
21
34
18
37
29
25
35
9
24
25
27
9
43
33
33
67
28
18
20
58
27
21
34
18
37
29
25
35
9
24
25
27
9
43
33
33
78
38
58
43
44
77
12
54
86
26
55
52
44
77
12
54
34
55
46
42
114
54
60
39
60
26
46
26
109
54
60
39
24
75
59
56
40
74
39
26
34
25
23
22
24
25
23
22
24
25
23
47
11
18
34
29
24
19
29
45
28
63
35
43
42
22
39
39
36
24
35
43
42
28
25
21
18
33
44
37
57
35
43
42
22
39
39
42
33
27
26
57
27
26
47
35
43
42
22
39
39
54
17
22
59
41
41
104
49
50
79
42
39
86
56
50
84

13
45
37
52
45
43
49
51
26
39
40
44
54
37
41
48
22
31
40
44
37
45
39
48
13
56
70
59
58
39
54
49
49
61
27
26
42
32
29
42
33
30
30
28
26
34
18
27
41
30
23
23
25
26
23
21
28
40
40
35
43
43
52
40
47
36
44
35
27
26
45
40
39
22
22
22
26
23
21
15
21
10
10
10
45
46
37
24
47
48
38
49
39
40
33
25
36
32
39
44
23
23
45
21
26
30
32
18
33
46
39
52
39
42
46
37
34
36
49
61
50
44
24
56
63
47
56
52
50
50
58
65
55
53
31
42
21
65
37
40
49
56
58
61
49
27
46
65
47
40
48
50
49
39
51
49
47
50
50
42
41
56
40
40
34
42
44
22
81
42
31
34
28
22
56
24
41
26
40
66
39
28
51
44
64
22
49
37
41
51
32
49
56
58
54
42
59
48
54
37
35
16
41
39
38
44
43
44
45
46
33
57
43
42
38
27
31
56
57
24
27
28
28
38
30
36
36
36
45
43
52
42
53
41
46
30
33
33
31
35
38
31
43
34
30
27
25
21
26
29
27
23
23
22
29
30
25
20
23
17
19
23
22
30
29
28
21
25
30
24
27
21
21
21
23
28
26
22
29
40
54
43
57
56
48
39
42
40
56
44
47
48
30
42
44
42
38
48
55
39
39
37
40
50
43
57
61
34
49
51
57
51
53
42
53
43
41
55
58
46
38
52
37
15
55
45
45
44
37
41
39
22
55
5

34
39
41
44
41
32
62
55
23
27
43
44
41
40
33
46
46
49
41
37
50
30
31
34
28
46
47
51
42
50
40
37
44
62
38
49
45
49
21
57
48
55
48
50
20
49
47
46
32
56
54
42
35
50
43
26
52
48
49
36
46
39
50
36
41
21
50
46
51
44
52
57
54
75
48
41
48
40
48
44
39
39
32
37
47
45
61
33
47
49
46
46
49
32
49
43
33
43
34
42
39
42
31
45
52
52
59
38
65
34
43
45
50
17
11
11
8
23
9
25
10
21
16
33
20
19
46
41
41
46
21
12
44
14
40
61
20
47
45
27
33
31
49
38
46
48
46
42
35
46
46
48
51
43
43
44
38
35
35
45
33
11
22
34
56
50
48
45
50
63
39
19
61
51
47
36
46
59
56
44
23
45
57
54
24
38
19
41
41
50
33
47
38
60
47
64
50
44
44
36
58
35
40
48
54
37
51
36
48
55
47
41
40
35
52
47
47
29
63
23
26
55
28
25
34
46
27
21
22
26
32
40
41
38
48
51
47
50
36
43
27
24
32
17
17
21
42
33
19
20
21
20
40
34
53
28
24
42
28
22
32
32
30
32
34
49
49
59
46
54
27
32
51
56
46
59
40
49
42
40
45
46
51
45
47
43
33
26
16
50
45
49
46
44
48
26
25
31
47
44
38
48
47
49
50
12
38
38
44
45
42
40
46
57
44
53
61
51
45
57
48
46
48
50
50
54
44
45
49
42
42
47
42
50


11
37
36
35
23
38
39
33
28
31
18
30
58
8
39
32
46
58
5
40
35
39
44
25
39
33
43
24
31
35
10
37
46
42
43
31
43
29
36
27
30
36
12
44
43
14
39
39
37
39
60
37
54
49
57
11
35
31
33
55
28
28
60
29
28
29
43
37
32
32
58
28
28
60
29
28
29
43
44
34
34
58
28
28
60
29
28
29
43
48
60
13
21
15
23
22
21
24
24
10
10
7
28
13
20
21
20
18
21
44
24
10
10
7
7
7
40
22
29
22
22
19
46
32
50
34
77
27
31
22
24
19
46
32
50
34
43
68
19
24
42
34
23
26
53
19
23
26
52
21
38
16
59
21
21
21
74
31
33
55
28
28
60
29
28
29
43
37
32
32
58
28
28
60
29
28
29
43
44
34
34
58
28
28
60
29
28
29
43
48
60
13
21
15
23
22
21
24
24
10
10
7
28
13
20
21
20
18
21
44
24
10
10
7
7
7
40
22
29
22
22
19
46
32
50
34
77
27
31
22
24
19
46
32
50
34
43
68
19
24
42
34
23
26
53
19
23
26
52
21
38
16
59
21
21
21
79
55
48
55
5
24
40
4
5
24
40
4
5
24
97
42
44
62
5
24
40
4
5
24
40
4
5
24
94
55
48
55
5
24
40
4
5
24
40
4
5
24
106
21
18
47
24
41
18
26
21
60
15
46
16
46
16
28
15
44
31
31
64
31
31
59
22
18
44
17
42
18
26
33
11
11
11
11
47
31
31
64
31
31
84
4

KeyboardInterrupt: 

# Step 3. 데이터 정제

### 3-1)정규표현식을 이용한 corpus 생성

In [4]:
import re
def preprocess_sentence(sentence):
    sentence = sentence.lower().strip() # 1
    sentence = re.sub(r"([?.!,¿])", r" \1 ", sentence) # 2
    sentence = re.sub(r'[" "]+', " ", sentence) # 3
    sentence = re.sub(r"[^a-zA-Z?.!,¿]+", " ", sentence) # 4
    sentence = sentence.strip() # 5
    sentence = '<start> ' + sentence + ' <end>' # 6
    return sentence

In [5]:
filtered_corpus = []

for sentence in raw_corpus:
    if len(sentence)==0: continue
    #if sentence[-1] ==":" : continue
    
    preprocessd_sentence = preprocess_sentence(sentence)
    filtered_corpus.append(preprocessd_sentence)
    
filtered_corpus[:10]

['<start> all of this and more is for you <end>',
 '<start> with love , sincerity and deepest care <end>',
 '<start> my life with you i share <end>',
 '<start> ever since i met you , baby <end>',
 '<start> i ve been wantin to lay you down <end>',
 '<start> but it s so hard to get you <end>',
 '<start> baby , when you never come around <end>',
 '<start> every day that you keep it away <end>',
 '<start> it only makes me want it more <end>',
 '<start> ooh baby , just say the word <end>']

### 3-2) 코퍼스를 텐서로 변환

In [6]:
def tokenize(corpus):
    tokenizer = tf.keras.preprocessing.text.Tokenizer(
        num_words = 12000,
        filters = ' ', #필터링은 이미 했음.
        oov_token = "<unk>"
    )
    tokenizer.fit_on_texts(corpus) #문자 데이터를 입력받아 리스트 형태로 리턴 
    tensor = tokenizer.texts_to_sequences(corpus) #텍스트 안의 단어들을 숫자의 시퀀스로 변환
    
    #입력데이터의 시퀀스 길이를 일정하게 맞춰준다.
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor,padding='post')
    print(tensor,tokenizer)
    return tensor,tokenizer

In [7]:
tensor,tokenizer = tokenize(filtered_corpus)

[[  2  24  19 ...   0   0   0]
 [  2  31  33 ...   0   0   0]
 [  2  13 102 ...   0   0   0]
 ...
 [  2  39  39 ...   0   0   0]
 [  2   5  22 ...   0   0   0]
 [  2  39  39 ...   0   0   0]] <keras_preprocessing.text.Tokenizer object at 0x7fcb9c742510>


In [8]:
for idx in tokenizer.index_word:
    print(idx, ":",tokenizer.index_word[idx])

1 : <unk>
2 : <start>
3 : <end>
4 : ,
5 : i
6 : the
7 : you
8 : and
9 : a
10 : to
11 : it
12 : me
13 : my
14 : in
15 : t
16 : s
17 : that
18 : on
19 : of
20 : .
21 : your
22 : m
23 : we
24 : all
25 : like
26 : is
27 : be
28 : for
29 : up
30 : so
31 : with
32 : can
33 : love
34 : know
35 : but
36 : just
37 : don
38 : no
39 : oh
40 : what
41 : got
42 : this
43 : ?
44 : get
45 : they
46 : when
47 : do
48 : she
49 : yeah
50 : now
51 : baby
52 : if
53 : go
54 : re
55 : he
56 : out
57 : was
58 : one
59 : !
60 : down
61 : ll
62 : want
63 : let
64 : there
65 : see
66 : come
67 : cause
68 : say
69 : her
70 : not
71 : at
72 : make
73 : time
74 : from
75 : back
76 : have
77 : are
78 : how
79 : never
80 : girl
81 : as
82 : man
83 : take
84 : right
85 : way
86 : wanna
87 : will
88 : im
89 : ain
90 : need
91 : ve
92 : tell
93 : here
94 : then
95 : u
96 : gonna
97 : who
98 : more
99 : some
100 : where
101 : feel
102 : life
103 : could
104 : too
105 : his
106 : been
107 : said
108 : good
109 : or
110 

782 : fucked
783 : treat
784 : almost
785 : prayer
786 : business
787 : knees
788 : cars
789 : sad
790 : keeps
791 : turned
792 : falling
793 : lift
794 : peace
795 : breathe
796 : kinda
797 : naked
798 : paid
799 : looks
800 : skin
801 : supposed
802 : anybody
803 : ur
804 : everywhere
805 : probably
806 : bobby
807 : piece
808 : fresh
809 : outside
810 : mirror
811 : spot
812 : singing
813 : callin
814 : takin
815 : funny
816 : anymore
817 : few
818 : hoe
819 : yah
820 : hang
821 : giving
822 : ran
823 : quick
824 : telling
825 : stone
826 : mary
827 : queen
828 : bridge
829 : homie
830 : thinkin
831 : playing
832 : grow
833 : dancing
834 : jump
835 : cross
836 : means
837 : least
838 : yellow
839 : kid
840 : booty
841 : carry
842 : magic
843 : rather
844 : cup
845 : fat
846 : ye
847 : wings
848 : near
849 : everyday
850 : job
851 : human
852 : needs
853 : devil
854 : birthday
855 : nowhere
856 : glad
857 : fake
858 : nights
859 : asked
860 : wife
861 : angel
862 : reasons
863 : tear

1448 : coast
1449 : split
1450 : began
1451 : build
1452 : sippin
1453 : blunts
1454 : mister
1455 : older
1456 : bound
1457 : evil
1458 : deserve
1459 : silly
1460 : east
1461 : willing
1462 : breathing
1463 : pockets
1464 : toast
1465 : ate
1466 : dough
1467 : rum
1468 : pimp
1469 : album
1470 : jolene
1471 : alert
1472 : pum
1473 : toy
1474 : taught
1475 : imagine
1476 : dad
1477 : cop
1478 : mack
1479 : threw
1480 : bedroom
1481 : lines
1482 : player
1483 : sip
1484 : explain
1485 : soldier
1486 : triple
1487 : bless
1488 : building
1489 : smalls
1490 : champagne
1491 : stole
1492 : bunch
1493 : private
1494 : climb
1495 : taken
1496 : lifetime
1497 : stood
1498 : blew
1499 : bee
1500 : boots
1501 : wherever
1502 : claim
1503 : blunt
1504 : murder
1505 : est
1506 : breakin
1507 : insane
1508 : barely
1509 : bleed
1510 : worse
1511 : hits
1512 : hmm
1513 : rough
1514 : flower
1515 : platinum
1516 : dangerous
1517 : nervous
1518 : fore
1519 : swallow
1520 : dee
1521 : sugar
1522 : st

2090 : cadillac
2091 : parties
2092 : person
2093 : bold
2094 : cousin
2095 : sings
2096 : rocked
2097 : pee
2098 : months
2099 : mission
2100 : staying
2101 : reality
2102 : note
2103 : rep
2104 : grows
2105 : stayed
2106 : mcs
2107 : alarm
2108 : sober
2109 : britney
2110 : outrageous
2111 : deeper
2112 : flames
2113 : fading
2114 : thy
2115 : temptation
2116 : revolution
2117 : choice
2118 : yard
2119 : cherry
2120 : mood
2121 : twist
2122 : third
2123 : pound
2124 : shadow
2125 : waves
2126 : rent
2127 : eatin
2128 : circle
2129 : trigger
2130 : march
2131 : month
2132 : indeed
2133 : further
2134 : legacy
2135 : puttin
2136 : especially
2137 : jet
2138 : mercedes
2139 : hater
2140 : simmer
2141 : henry
2142 : jordan
2143 : jumpman
2144 : usually
2145 : bother
2146 : delirious
2147 : mothers
2148 : direction
2149 : magazine
2150 : putting
2151 : eggs
2152 : plays
2153 : gentlemen
2154 : motion
2155 : sigh
2156 : biggest
2157 : moonlight
2158 : amen
2159 : cindy
2160 : mix
2161 : nu

2687 : beer
2688 : habits
2689 : outro
2690 : apologies
2691 : pissed
2692 : promises
2693 : illusion
2694 : turnin
2695 : se
2696 : eighteen
2697 : ignore
2698 : maria
2699 : rick
2700 : emotion
2701 : tiger
2702 : alexander
2703 : ross
2704 : minaj
2705 : perdiendo
2706 : careful
2707 : juice
2708 : laughter
2709 : destiny
2710 : vous
2711 : et
2712 : disease
2713 : tap
2714 : smiles
2715 : rubber
2716 : boogie
2717 : jughead
2718 : sucker
2719 : stepped
2720 : peak
2721 : ace
2722 : rode
2723 : speakin
2724 : eyed
2725 : esta
2726 : mystery
2727 : lamb
2728 : robbed
2729 : vest
2730 : charm
2731 : maid
2732 : burst
2733 : chief
2734 : spoken
2735 : basement
2736 : picking
2737 : mud
2738 : strapped
2739 : millionaire
2740 : wreck
2741 : replace
2742 : purse
2743 : blocks
2744 : cocked
2745 : jokes
2746 : criminal
2747 : superstar
2748 : showin
2749 : woulda
2750 : um
2751 : rapping
2752 : engine
2753 : lighters
2754 : digger
2755 : movement
2756 : princess
2757 : fades
2758 : bro
27

...

24669 : flamed
24670 : belting
24671 : sowing
24672 : awooing
24673 : sirs
24674 : hoarse
24675 : stealer
24676 : dingty
24677 : diddledy
24678 : gilly
24679 : dairy
24680 : snips
24681 : medley
24682 : giblets
24683 : flapped
24684 : herrings
24685 : tups
24686 : twee
24687 : lengthening
24688 : lasses
24689 : nightgown
24690 : thriven
24691 : kyloe
24692 : broth
24693 : repartee
24694 : sacaradown
24695 : codlin
24696 : codlins
24697 : boar
24698 : goodly
24699 : pecks
24700 : thereof
24701 : mounted
24702 : maccaroni
24703 : playfellows
24704 : halfpenny
24705 : ploughed
24706 : peppercorn
24707 : harrowed
24708 : bramble
24709 : reaped
24710 : banbury
24711 : twopenny
24712 : prod
24713 : arithmetic
24714 : vexation
24715 : varied
24716 : woodcock
24717 : thrice
24718 : candlestick
24719 : tuffet
24720 : curds
24721 : whey
24722 : roundelay
24723 : trowel
24724 : rook
24725 : tarries
24726 : heaved
24727 : shepherdess
24728 : nan
24729 : paddocks
24730 : benison
24731 : chandler
24

25245 : penalized
25246 : khlaed
25247 : alphabetical
25248 : lucas
25249 : promoters
25250 : roguish
25251 : decoded
25252 : quotin
25253 : everyting
25254 : cruised
25255 : oozed
25256 : nudes
25257 : uhn
25258 : badness
25259 : intelligence
25260 : whatsapp
25261 : bbm
25262 : wanton
25263 : shigady
25264 : digady
25265 : allot
25266 : migady
25267 : biggady
25268 : sik
25269 : purpo
25270 : fendy
25271 : groopie
25272 : chingalling
25273 : oochi
25274 : oouchi
25275 : wallin
25276 : probleme
25277 : lok
25278 : ott
25279 : vaga
25280 : cenemar
25281 : dickin
25282 : apres
25283 : dingin
25284 : apred
25285 : energeny
25286 : outsa
25287 : streight
25288 : putsa
25289 : tung
25290 : gobblin
25291 : chik
25292 : dik
25293 : jeopardy
25294 : medicate
25295 : cappuccino
25296 : vodkas
25297 : reaping
25298 : rewards
25299 : dollas
25300 : neil
25301 : balboa
25302 : uncalled
25303 : whoopie
25304 : recoil
25305 : numeric
25306 : incoherent
25307 : aborting
25308 : urn
25309 : hashish
2

25907 : chubbs
25908 : lickwood
25909 : requested
25910 : mortgages
25911 : unsuccessful
25912 : paramedic
25913 : californication
25914 : ripley
25915 : satanic
25916 : blaine
25917 : pitching
25918 : underhanded
25919 : outstanding
25920 : spraying
25921 : heaters
25922 : febreeze
25923 : tl
25924 : tsx
25925 : gelo
25926 : juugs
25927 : prant
25928 : ginobili
25929 : fadeaway
25930 : mutombo
25931 : residual
25932 : aubrey
25933 : vouch
25934 : oppers
25935 : ethics
25936 : alfredo
25937 : pasta
25938 : elses
25939 : immigration
25940 : printing
25941 : whoopty
25942 : franchise
25943 : percies
25944 : mergers
25945 : urgent
25946 : pinkett
25947 : rosetta
25948 : smiddoke
25949 : skeptic
25950 : widescreen
25951 : roja
25952 : loca
25953 : chargi
25954 : tropics
25955 : cyaan
25956 : unrealistic
25957 : syd
25958 : unforgiven
25959 : tropical
25960 : amalfi
25961 : adiios
25962 : functioning
25963 : mufuckin
25964 : taylors
25965 : blammers
25966 : scout
25967 : yap
25968 : katia
2

26573 : refery
26574 : cryn
26575 : sexing
26576 : shuttle
26577 : monotone
26578 : naggers
26579 : natin
26580 : wiki
26581 : ific
26582 : suntrust
26583 : chihuahua
26584 : rottweila
26585 : metta
26586 : regulars
26587 : australian
26588 : miniskirt
26589 : raris
26590 : eses
26591 : nitty
26592 : kidnappin
26593 : ebk
26594 : ebt
26595 : investigations
26596 : domination
26597 : opposition
26598 : bullseye
26599 : muny
26600 : sext
26601 : samsung
26602 : rabbi
26603 : brisbane
26604 : chucker
26605 : ducka
26606 : suppa
26607 : assess
26608 : chum
26609 : chummy
26610 : shotta
26611 : woodies
26612 : russel
26613 : vince
26614 : mcmahon
26615 : cm
26616 : snowflow
26617 : aspens
26618 : poconos
26619 : calogero
26620 : tampering
26621 : lifer
26622 : fashionistist
26623 : smugglin
26624 : chow
26625 : leprechaun
26626 : cleo
26627 : skio
26628 : konichiwa
26629 : leprachaun
26630 : bff
26631 : ballsack
26632 : motivators
26633 : marquis
26634 : walkie
26635 : talkie
26636 : inters

27072 : portia
27073 : degeneres
27074 : ventolin
27075 : ritalin
27076 : binge
27077 : mayer
27078 : alfalfa
27079 : shrinked
27080 : rapes
27081 : assaults
27082 : diabolic
27083 : tusk
27084 : disgusts
27085 : mushed
27086 : worcestershire
27087 : dammed
27088 : dorado
27089 : vinyl
27090 : precision
27091 : maaco
27092 : waco
27093 : introducing
27094 : napalm
27095 : shaun
27096 : caking
27097 : mothefucking
27098 : outty
27099 : perfectness
27100 : divorcing
27101 : helplessness
27102 : vindictive
27103 : az
27104 : ingrained
27105 : shitstain
27106 : unlearn
27107 : pondered
27108 : artform
27109 : pluggin
27110 : kabillion
27111 : spawns
27112 : recycling
27113 : indulgent
27114 : uprise
27115 : supplies
27116 : eggshells
27117 : nips
27118 : senna
27119 : indy
27120 : soho
27121 : wynonna
27122 : fairies
27123 : sketchy
27124 : headfirst
27125 : brunt
27126 : belongings
27127 : fixable
27128 : changeable
27129 : rightfully
27130 : deteriorating
27131 : overwhelming
27132 : sco

In [12]:
print(tensor[50])

[ 2 14 33  3  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0

### 3-3) corpus 텐서를 tf.data.Dataset객체로 변환

In [20]:
src_input = tensor[:,:-1]
tgt_input = tensor[:,1:]

In [21]:
BUFFER_SIZE = len(src_input)
BATCH_SIZE = 256
steps_per_epoch = len(src_input) // BATCH_SIZE

VOCAB_SIZE = tokenizer.num_words +1

dataset = tf.data.Dataset.from_tensor_slices((src_input,tgt_input))
dataset = dataset.shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE,drop_remainder=True)
dataset

2022-12-30 15:46:29.063279: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


<BatchDataset shapes: ((256, 346), (256, 346)), types: (tf.int32, tf.int32)>

# Step 4. 평가 데이터셋 분리

In [22]:
from sklearn.model_selection import train_test_split

X = tensor[:,:-1]
y = tensor[:,1:]
enc_train, enc_val, dec_train, dec_val = train_test_split(X,y,test_size = 0.2)

# Step 5. 인공지능 만들기

In [23]:
class TextGenerator(tf.keras.Model):
    def __init__(self, vocab_size, embedding_size, hidden_size):
        super().__init__()
        # Embedding 레이어, 2개의 LSTM 레이어, 1개의 Dense 레이어로 구성되어 있다.
        # Embedding 레이어는 단어 사전의 인덱스 값을 해당 인덱스 번째의 워드 벡터로 바꿔준다.
        # 이 워드 벡터는 의미 벡터 공간에서 단어의 추상적 표현으로 사용된다. 
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_size) 
        self.rnn_1 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)  
        self.rnn_2 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.linear = tf.keras.layers.Dense(vocab_size)
        
    def call(self, x):
        out = self.embedding(x)
        out = self.rnn_1(out)
        out = self.rnn_2(out)
        out = self.linear(out)
        
        return out

In [24]:
embedding_size = 256
hidden_size = 1024
model = TextGenerator(tokenizer.num_words+1, embedding_size,hidden_size)

In [25]:
for src_sample, tgt_sample in dataset.take(1): break
    
model(src_sample)

<tf.Tensor: shape=(256, 346, 12001), dtype=float32, numpy=
array([[[ 3.66443419e-06,  7.89180340e-05, -1.48993422e-04, ...,
         -1.01360216e-04,  5.74639635e-05,  1.70913554e-04],
        [-1.90253311e-04,  1.57734437e-04, -8.65435322e-06, ...,
         -7.86930905e-05,  5.66915842e-05,  1.91368832e-04],
        [-3.33104079e-04,  4.48303501e-04,  2.42241476e-05, ...,
         -1.09638186e-05,  1.25387916e-04,  8.93990509e-05],
        ...,
        [-4.97871591e-03,  3.80400429e-03,  3.88736883e-03, ...,
         -2.65961513e-04,  3.55663337e-03,  3.07520805e-03],
        [-4.97871591e-03,  3.80400452e-03,  3.88736883e-03, ...,
         -2.65961309e-04,  3.55663337e-03,  3.07520828e-03],
        [-4.97871544e-03,  3.80400429e-03,  3.88736837e-03, ...,
         -2.65961251e-04,  3.55663360e-03,  3.07520805e-03]],

       [[ 3.66443419e-06,  7.89180340e-05, -1.48993422e-04, ...,
         -1.01360216e-04,  5.74639635e-05,  1.70913554e-04],
        [ 8.79106738e-05,  1.44224032e-05, -

In [26]:
model.summary()

Model: "text_generator"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        multiple                  3072256   
_________________________________________________________________
lstm (LSTM)                  multiple                  5246976   
_________________________________________________________________
lstm_1 (LSTM)                multiple                  8392704   
_________________________________________________________________
dense (Dense)                multiple                  12301025  
Total params: 29,012,961
Trainable params: 29,012,961
Non-trainable params: 0
_________________________________________________________________


In [ ]:
optimizer = tf.keras.optimizers.Adam() # Adam은 현재 가장 많이 사용하는 옵티마이저이다. 자세한 내용은 차차 배운다.
loss = tf.keras.losses.SparseCategoricalCrossentropy( # 훈련 데이터의 라벨이 정수의 형태로 제공될 때 사용하는 손실함수이다.
    from_logits=True, # 기본값은 False이다. 모델에 의해 생성된 출력 값이 정규화되지 않았음을 손실 함수에 알려준다. 즉 softmax함수가 적용되지 않았다는걸 의미한다. 
    reduction='none'  # 기본값은 SUM이다. 각자 나오는 값의 반환 원할 때 None을 사용한다.
)
# 모델을 학습시키키 위한 학습과정을 설정하는 단계이다.
model.compile(optimizer=optimizer, loss=loss) # 손실함수와 훈련과정을 설정했다.
model.fit(dataset, epochs=10) # 만들어둔 데이터셋으로 모델을 학습한다. 30번 학습을 반복하겠다는 의미다.

Epoch 1/10


2022-12-30 15:50:47.793134: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


  6/687 [..............................] - ETA: 52:19:15 - loss: 4.2052